In [1262]:
!pip install scikit-learn

In [1263]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore

In [1284]:
# reading in the major csv files, to work on them and edit the datasets
df_sea = pd.read_csv(r"C:\Users\shero\Downloads\NBA Project\Copy of seasons(updated).csv")
df_cpl = pd.read_csv(r"C:\Users\shero\Downloads\NBA Project\Copy of players(updated).csv")
df_phw = pd.read_csv(r"C:\Users\shero\Downloads\NBA Project\Copy of players_height_weight(updated).csv")
df_ndd = pd.read_csv(r"C:\Users\shero\Downloads\NBA Project\nba_draft_data.csv")

# reading in sql edited csv files, to work on them and edit the datasets
df_ms = pd.read_csv(r"C:\Users\shero\Downloads\NBA Project\main_stats.csv")
df_pl = pd.read_csv(r"C:\Users\shero\Downloads\NBA Project\players.csv")
df_pm = pd.read_csv(r"C:\Users\shero\Downloads\NBA Project\players_measure.csv")
df_as = pd.read_csv(r"C:\Users\shero\Downloads\NBA Project\advanced_stats_temp.csv")

#df_sea
#df_cpl
#df_phw
#df_ndd

#df_ms
#df_pl
#df_pm
#df_as

In [1286]:
# looking at the datatypes of each dataset, to get a better understanding of the dataset and how I should edit/analyze the data
#df_sea.info()
#df_cpl.info()
#df_phw.info()
#df_ndd.info()

#df_ms.info()
#df_pl.info()
#df_pm.info()
#df_as.info()

In [1288]:
#updating the positions to match each other across datasets, to make it easier to merge them 

df_sea['Pos'] = df_sea['Pos'].replace({'PG':'Guard', 'SG':'Guard', 'SF': 'Wing', 'PF': 'Big', 'C':'Big'})
df_cpl['position'] = df_cpl['position'].replace({'PG':'Guard', 'SG':'Guard', 'SF': 'Wing', 'PF': 'Big', 'C':'Big'})
df_as['Pos'] = df_as['Pos'].replace({'PG':'Guard', 'SG':'Guard', 'SF': 'Wing', 'PF': 'Big', 'C':'Big'})

#updating columns name to merge the dataets
df_sea = df_sea.rename(columns={"Player":"player"})
df_cpl = df_cpl.rename(columns={"name":"player"})
df_phw = df_phw.rename(columns={"Player":"player"})

df_pl = df_pl.rename(columns={"name": "player"})
df_as = df_as.rename(columns={"Player":"player"})

df_ndd = df_ndd.rename(columns={'pts_per_g':'ppg'})
df_ndd = df_ndd.rename(columns={'fg_pct':'fg'})
df_ndd = df_ndd.rename(columns={'fg3_pct':'3p'})
df_ndd = df_ndd.rename(columns={'ft_pct':'ft'})
df_ndd = df_ndd.rename(columns={'mp_per_g':'mpg'})
df_ndd = df_ndd.rename(columns={'trb_per_g':'rbg'})
df_ndd = df_ndd.rename(columns={'ast_per_g':'asg'})

In [1290]:
# merge multiple datasets, to get one dataset that has all I need to start analyzing the data

#df_full_player_demos
df_fpw = df_ms.merge(df_pl, on = 'player', how = "outer").merge(
    df_pm, on = 'player', how = "outer").merge(
    df_as, on = 'player', how = "outer").merge(
    df_ndd, on = 'player', how = "outer")

# shows all columns, so I can select the ones I want
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

df_fpw = df_fpw.rename(columns= {'college_x': 'college', 'g_x':'g', '2p_x':'2p',
'ppg_x':'ppg', 'asg_x':'asg','rbg_x':'rbg','fg_x':'fg','ft_x':'ft', 'mpg_x':'mpg'})

# filtering the columns desired
df_fpw = df_fpw[['player','pick_overall', 'college', 'birth_state','height','weight', 'position', 
"ppg", 'asg', 'rbg', 'fg', '3P','2p', 'ft', 'stl', 'blk', 'g', 'gs', 'mpg']]

# cleaning up the merged column names

#df_fpw.head(1)

In [1292]:
# remove duplicates from datasets
df_fpw = df_fpw.groupby('player', as_index = False).agg({ 'pick_overall':'first','college':'first', 'height':'first', 'weight':'first', 
    'position':'first', 'ppg':'mean', 'asg':'mean', 'rbg':'mean', 'fg':'mean', 
    '3P':'mean', '2p': 'mean', 'ft':'mean', 'stl':'mean', 'blk':'mean','g':'mean','gs':'mean', 'mpg':'mean'})

df_as = df_as.groupby('player', as_index = False).agg({
    'Pos':'first','PER':'mean', 'TS':'mean', 'USG':'mean', 
    'WS':'mean', 'FG':'mean', '3p':'mean', '2p':'mean', 'eFG%':'mean', 
    'FT':'mean'})

#df_sea = df_sea.groupby('player', as_index = False).agg({  'Pos': 'first', 'PER': 'mean',  'TS%': 'mean', 'USG%': 'mean', 'WS': 'mean', 'FG%':'mean',
    #'3PPercentage':'mean','2p/Dis':'mean','eFG%':'mean','FT/Dis':'mean','RBG':'mean','ASG':'mean','STL/Dis':'mean','BLK/Dis':'mean','TPG':'mean',
    #'PFG':'mean','PPG':'mean'})



In [1294]:
#filling NaN values with 0, to get a cleaner looking dataset
df_sea = df_sea.fillna(0)
df_cpl = df_cpl.fillna(0)
df_phw = df_phw.fillna(0)
df_ndd = df_ndd.fillna(0)

df_ms = df_ms.fillna(0)
df_pl = df_pl.fillna(0)
df_pm = df_pm.fillna(0)
df_as = df_as.fillna(0)

df_fpw = df_fpw.fillna(0)
df_pick = df_pick.fillna(0)

In [1296]:
# remove rows the have no major stats data

df_fpw = df_fpw[(df_fpw['g'] > 0)]

# assigning positions to rows that don't have them by height
def assign_position(height):
    if height < 6.3:
        return 'Guard'
    elif 6.3 < height < 6.8:
        return 'Wing'
    else:
        return 'Big'
        
df_fpw['position'] = df_fpw.apply(lambda row: assign_position(row['height']) if row['position'] == 0 else row['position'], axis =1)

In [1298]:
# position based analysis

# getting descriptive stats by position -------------------------
Pos_stats = df_fpw.groupby('position')[['ppg','asg','rbg','stl','blk']].agg(['mean', 'median', 'min', 'max'])

# position counts ---------------------------
Pos_count = df_fpw['position'].value_counts()

# creating a box plot to visualize data -----------------------

#plt.figure(figsize = (10,6))
#sns.boxplot(x=df_fpw['position'], y= df_fpw['ppg'], palette='Set2')
#plt.title('Dis of ppg by pos')
#plt.xlabel('POS')
#plt.ylabel('PPG')
#plt.show()


# shooting stats by pos

#Shooting = df_fpw.groupby('position').agg({ '3P':'mean','2p':'mean','fg': 'mean', 'ft': 'mean'})
    
# bar chart for shooting stats-----------------------

#Shooting.plot(kind = 'bar', figsize=(12,5), colormap = 'coolwarm')
#plt.title('Shooting Efficiency by Position')
#plt.xlabel('Position')
#plt.xticks(rotation=0)
#plt.ylabel('Shooting Percentage')
#plt.show()

# separate starters and bench players ---------------------
# Starters = df_fpw[(df_fpw['gs'] >= 41)]
# Bench = df_fpw[(df_fpw['gs'] < 41)]


#shoot_starters = df_fpw[(df_fpw['gs'] >= 41)].groupby('position').agg({ '3P':'mean','2p':'mean','fg': 'mean', 'ft': 'mean'})
#shoot_bench = df_fpw[(df_fpw['gs'] < 41)].groupby('position').agg({ '3P':'mean','2p':'mean','fg': 'mean', 'ft': 'mean'})

# z-score -----------------------
# df_fpw['ppg_z'] = df_fpw.groupby('position')['ppg'].transform(zscore)

# outliers = df_fpw[df_fpw['ppg_z'] > 2][['player', 'position', 'ppg', 'ppg_z']]

# time series ------------------------
#Trend_Pos_stats = df_sea.groupby(['Year','Pos'])[['ppg','asg','rbg','stl','blk']].agg(['mean'}).unstack()

#Trend_Pos_PPG = df_sea.groupby(['Year','Pos'])[['PPG']].agg(['mean']).unstack()

#Trend_Pos_PPG.plot(figsize=(10,5), marker = 'o')
#plt.title('PPG Stats Trends by position over time')
#plt.xlabel('Year')
#plt.ylabel('Major Stats')
#plt.legend(title='Position')
#plt.show()

In [1468]:
# Draft pick analysis

# get dataset for pick analysis
df_pick = df_ndd.merge(df_as, on='player', how = 'left').merge(df_pm, on='player', how = 'left')
df_pick = df_pick.fillna(0)


# filter columns
df_pick = df_pick[['player','pick_overall', 'team_id','college_name','height','weight', 'seasons', 'Pos', 
"ppg", 'asg', 'rbg', 'fg', '3p_y','2p', 'ft', 'g', 'USG', 'PER', 'ws', 'bpm', 'vorp', 'TS', 'birth_state']]

# filter and groupby pick
pick = df_pick[df_pick['pick_overall'] <= 60]
pick_ppg = pick.groupby('pick_overall', as_index = False)['ppg'].mean()

df_pick_p = df_pick[(df_pick['PER'] != 0) & (df_pick['pick_overall'] <= 60)]
pick_per = df_pick_p.groupby('pick_overall', as_index = False)['PER'].mean()

# visualize the data

# scatter plot of ppg by pick
#plt.scatter(pick_ppg['pick_overall'], pick_ppg['ppg'])
#plt.xlabel('pick_overall')
#plt.ylabel('PPG')
#plt.title('PPG Stats Trends by pick overall')
#plt.show()

# scatter of PER by pick
#plt.scatter(pick_per['pick_overall'], pick_per['PER'])
#plt.xlabel('pick_overall')
#plt.ylabel('PER')
#plt.title('PER Trends by pick overall')
#plt.show()

# see which teams draft the best scorers

team = pick_p
team_p = team.groupby('team_id', as_index = False)[['ppg', 'PER']].mean()
team_p = team_p.sort_values(by='ppg', ascending = False)

#team_p

# updating players without a position based on height
def assign_position(height):
    if height < 6.3:
        return 'Guard'
    elif 6.3 < height < 6.8:
        return 'Wing'
    else:
        return 'Big'
        
df_pick['Pos'] = df_pick.apply(lambda row: assign_position(row['height']) if row['Pos'] == 0 else row['Pos'], axis =1)

#sns.boxplot(x=df_pick['Pos'], y = df_pick['ppg'], hue=pd.cut(df_pick['pick_overall'], bins = [1, 10, 30, 60]))

# longevity

#sns.lineplot(x=pick['pick_overall'], y=pick['seasons'])
#plt.title('Seasons Played by Pick Number')
#plt.show()

# bust rate
def cat_pick(pick_):
    if pick_ <= 10:
        return 'Top 10'
    elif pick_ <= 20:
        return 'Mid 1st round'
    elif pick_ <= 30:
        return 'Late 1st round'
    else:
        return '2nd round'

df_pick['pick_range'] = df_pick['pick_overall'].apply(cat_pick)
pick_order = ['Top 10', 'Mid 1st round', 'Late 1st round', '2nd round']

df_pick['PER'] = df_pick.apply(
    lambda row: df_pick[(df_pick['Pos'] == row['Pos']) & (df_pick['pick_range'] == row['pick_range'])]['PER'].median() 
    if row['PER'] == 0 else row['PER'], axis=1 )
#df_pick['pick_range'] = pd.Categorical(df_pick['pick_range'], categories = pick_order, ordered = True)


#bust_rates = df_pick.groupby('pick_range')['bust'].mean().reset_index()
#bust_rates.columns = ['pick_range', 'bust_rate']
#bust_rates.sort_values(by='bust_rate', ascending = True)


#plt.figure(figsize=(8,5))
#sns.barplot(x=bust_rates['pick_range'], y=bust_rates['bust_rate'] * 100, palette = 'coolwarm', order = pick_order)
#plt.xlabel('Draft Pick Range')
#plt.ylabel('Bust Rate')
#plt.title('NBA draft bust rates by pick range')
#plt.ylim(0,70)
#plt.show()

# can show the bust rate by position and height

# use z-score to see which players outperformed and underpeformed
df_pick['per_z'] = df_pick.groupby('pick_range')['PER'].transform(lambda x: zscore (x, ddof=1))

def perf_cat(z):
    if z > 1.5:
        return 'Overperformed'
    elif z < -1.5:
        return 'Underperformed'
    else:
        return 'Average'

df_pick['per_p'] = df_pick['per_z'].apply(perf_cat)
per_filter = df_pick[df_pick['PER'] != 0]
df_pick_z = per_filter[['player', 'pick_overall', 'pick_range', 'PER', 'per_z', 'per_p']]

ops = df_pick_z[(df_pick_z['per_p'] == 'Overperformed')]
#ops

df_pick['bust'] = (
(df_pick['ppg'] < df_pick.groupby('pick_range')['ppg'].transform('mean') - 1.5 * df_pick.groupby('pick_range')['ppg'].transform('std')) | \
(df_pick['PER'] < df_pick.groupby('pick_range')['PER'].transform('mean') - 1.5 * df_pick.groupby('pick_range')['PER'].transform('std')) | \
(df_pick['seasons'] < 3))

df_pick['star'] = (
(df_pick['ppg'] > df_pick.groupby('pick_range')['ppg'].transform('mean') + 1.5 * df_pick.groupby('pick_range')['ppg'].transform('std')) &
(df_pick['PER'] > df_pick.groupby('pick_range')['PER'].transform('mean') + 1.5 * df_pick.groupby('pick_range')['PER'].transform('std')) &
(df_pick['PER'] != 0) &
(df_pick['seasons'] > 3))
# extra- college, birth state, more height , more pos 


In [1455]:
# Correlation analysis

#
corr_pick = per_filter[['pick_overall', 'ppg', 'TS' ,'PER', 'ws', 'bpm', 'vorp', 'USG']]

corr_m = corr_pick.corr()

# 
corr_pos = per_filter[['Pos', 'ppg', 'TS' ,'PER', 'ws', 'bpm', 'vorp', 'USG']]

#corr_m2 = corr_pos.corr()



In [1458]:
# Create archetypes by clustering

stats = per_filter[['ppg', 'PER', 'TS', 'USG', 'ws', 'bpm', 'vorp']]

df_clus = stats.copy()

scaler = StandardScaler()
df_clus_scal = scaler.fit_transform(df_clus)

kmeans = KMeans(n_clusters = 3, random_state = 42, n_init = 10)
df_clus['cluster'] = kmeans.fit_predict(df_clus_scal)

#df_clus['cluster'].value_counts()

#df_clus.groupby('cluster')[['ppg', 'PER', 'TS', 'USG', 'ws', 'bpm', 'vorp']]

#plt.figure(figsize = (10,6))
#sns.scatterplot(x=df_clus['ppg'], y = df_clus['PER'], hue =df_clus['cluster'], palette = 'viridis')
#plt.xlabel('PPG')
#plt.ylabel('PER')
#plt.title('NBA Player Cluster')
#plt.show()

In [1462]:
# export datasets to excel files

#df_sea.to_excel(r"C:\Users\shero\Downloads\sea.xlsx", index= False)
#df_cpl.to_excel(r"C:\Users\shero\Downloads\cpl.xlsx", index= False)
#df_phw.to_excel(r"C:\Users\shero\Downloads\phw.xlsx", index= False)
#df_ndd.to_excel(r"C:\Users\shero\Downloads\ndd.xlsx", index= False)

#df_ms.to_excel(r"C:\Users\shero\Downloads\ms.xlsx", index= False)
#df_pl.to_excel(r"C:\Users\shero\Downloads\pl.xlsx", index= False)
#df_pm.to_excel(r"C:\Users\shero\Downloads\pm.xlsx", index= False)
#df_as.to_excel(r"C:\Users\shero\Downloads\as.xlsx", index= False)

#df_fpw.to_excel(r"C:\Users\shero\Downloads\fpw.xlsx", index= False)
df_pick.to_excel(r"C:\Users\shero\Downloads\pick.xlsx", index= False)
#df_pick_p.to_excel(r"C:\Users\shero\Downloads\pick_p.xlsx", index= False)

In [1470]:
df_fpw_gs = df_fpw[(df_fpw['gs'].isna() | (df_fpw['gs'])) == 0 & (df_fpw['g'] == 0)]